# Homework 2

Importo los modulos necesarios

In [30]:
import requests
import numpy as np

from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models

## Q1. Embedding the query

In [30]:
def embed_data(text, model_embedding = 'jinaai/jina-embeddings-v2-small-en'):
    embedding_model = TextEmbedding(model_name=model_embedding)

    if type(text) == str:
        text = [text]
    
    result_embed = embedding_model.embed(text)    
    return list(result_embed)

In [31]:
text = 'I just discovered the course. Can I join now?'
result = embed_data(text)

In [32]:
result[0].shape

(512,)

In [33]:
result[0].min()

np.float64(-0.11726373885183883)

## Q2. Cosine similarity with another vector

`np.linalg.norm(q)`: calcula la norma euclidiana (también llamada norma L2) del vector q.

In [36]:
q = result[0]
np.linalg.norm(q)

np.float64(1.0)

Si el resultado es `1`, eso implica que el vector `q` es un vector normalizado, es decir, tiene una longitud (o magnitud) de exactamente 1.

Esto es muy común en el contexto de búsqueda semántica o embeddings, donde los modelos (como `FastEmbed`, `SentenceTransformers`, `OpenAI embeddings`, etc.) normalizan automáticamente sus vectores de salida. Esto se hace para que la similitud coseno entre vectores pueda calcularse simplemente con el producto punto.

In [37]:
q.dot(q)

np.float64(1.0000000000000002)

In [44]:
doc = 'Can I still join the course after the start date?'
result_2 = embed_data(doc)

In [45]:
r = result_2[0]
r.min()

np.float64(-0.12396320482168117)

In [46]:
q.dot(r)

np.float64(0.9008528895674548)

## Q3. Ranking by cosine

In [47]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [50]:
len(documents)

5

In [49]:
list_docs = [ doc['text'] for doc in documents]
len(list_docs)

5

In [54]:
result_3 = embed_data(list_docs)
type(result_3)

list

In [58]:
matriz_v = np.array(result_3)

In [57]:
matriz_v.shape

(5, 512)

In [68]:
dot_product = matriz_v.dot(q)
dot_product.max()

np.float64(0.8182378150042889)

In [69]:
[(index, value) for index, value in enumerate(dot_product)]

[(0, np.float64(0.7629684696540238)),
 (1, np.float64(0.8182378150042889)),
 (2, np.float64(0.8085397398734189)),
 (3, np.float64(0.7133079015686243)),
 (4, np.float64(0.73044992343336))]

In [70]:
[(index, value) for index, value in enumerate(dot_product) if value == dot_product.max()]

[(1, np.float64(0.8182378150042889))]

## Q4. Ranking by cosine, version two

In [106]:
list_docs_v2 = [f"{doc['question']} {doc['text']}" for doc in documents]
len(list_docs_v2)

5

In [107]:
result_4 = embed_data(list_docs_v2)
type(result_4)

list

In [108]:
matriz_v2 = np.array(result_4)

In [109]:
matriz_v2.shape

(5, 512)

In [110]:
dot_product_2 = matriz_v2.dot(q)
dot_product_2.max()

np.float64(0.8514543236908068)

In [111]:
[(index, value) for index, value in enumerate(dot_product_2)]

[(0, np.float64(0.8514543236908068)),
 (1, np.float64(0.843659415911307)),
 (2, np.float64(0.8408287048502558)),
 (3, np.float64(0.7755157969663907)),
 (4, np.float64(0.8086007795043937))]

In [112]:
[(index, value) for index, value in enumerate(dot_product_2) if value == dot_product_2.max()]

[(0, np.float64(0.8514543236908068))]

### Q3 vs Q4
Mejora de Similitud al Usar `question + text` en Embeddings

#### ¿Qué se hizo?

Se modificó la forma de generar el contenido sobre el cual se aplican los embeddings. En lugar de usar únicamente el campo `text` (respuesta), se concatenaron la `question` (pregunta) y el `text` en un solo string por cada documento.

Esto generó una nueva lista de textos enriquecidos que combina pregunta y respuesta en una sola secuencia.

In [113]:
f'Q3 max:{dot_product.max()} VS Q4 max:{dot_product_2.max()} is equal = {dot_product.max()==dot_product_2.max()}'

'Q3 max:0.8182378150042889 VS Q4 max:0.8514543236908068 is equal = False'

#### Comparación de Resultados

Se compararon los valores de similitud coseno obtenidos en dos escenarios:

| Método                      | Campo utilizado              | Máxima similitud obtenida |
|----------------------------|------------------------------|----------------------------|
| Solo respuesta (`text`)    | `text`                       | 0.8182                     |
| Pregunta + respuesta (`v2`) | `question + text`            | 0.8514                     |

> Las similitudes están expresadas como `np.float64`.

#### ¿Por qué mejora el resultado?

Al usar ambos campos (`question` y `text`), se enriquece el contexto semántico del contenido. Esto permite que el modelo de embeddings:
- Capte mejor la intención del documento.
- Genere representaciones vectoriales más informativas.
- Aumente la similitud con una pregunta de consulta al incluir también el tipo de pregunta esperada.

El resultado es una mayor similitud coseno y, por ende, una mejor recuperación de información.

Este cambio simple puede tener un impacto considerable en la calidad de las respuestas recuperadas.


## Q5. Selecting the embedding model

In [74]:
list_models = [{'model': doc['model'], 'dim': doc['dim']} for doc in TextEmbedding.list_supported_models()]
list_models[:2]

[{'model': 'BAAI/bge-base-en', 'dim': 768},
 {'model': 'BAAI/bge-base-en-v1.5', 'dim': 768}]

In [81]:
list_models_sorted = sorted(list_models, key=lambda x: x['dim'])
list_models_sorted[:10]

[{'model': 'BAAI/bge-small-en', 'dim': 384},
 {'model': 'BAAI/bge-small-en-v1.5', 'dim': 384},
 {'model': 'snowflake/snowflake-arctic-embed-xs', 'dim': 384},
 {'model': 'snowflake/snowflake-arctic-embed-s', 'dim': 384},
 {'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384},
 {'model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
  'dim': 384},
 {'model': 'BAAI/bge-small-zh-v1.5', 'dim': 512},
 {'model': 'Qdrant/clip-ViT-B-32-text', 'dim': 512},
 {'model': 'jinaai/jina-embeddings-v2-small-en', 'dim': 512},
 {'model': 'BAAI/bge-base-en', 'dim': 768}]

## Q6. Indexing with qdrant

In [29]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [38]:
COLLECTION_NAME = "zoomcamp-llm-hw2"
MODEL_SELECT = 'BAAI/bge-small-en'
SIZE_MODEL_SELECT = client.get_embedding_size(model_select)
SIZE_MODEL_SELECT

384

In [32]:
client = QdrantClient("http://localhost", port=6333)

In [39]:
if client.collection_exists(collection_name):
    client.delete_collection(collection_name)
    
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=models.VectorParams(
        size=SIZE_MODEL_SELECT, 
        distance=models.Distance.COSINE
    ),
    timeout=60
)

print(f"Colección '{COLLECTION_NAME}' creada exitosamente.")

Colección 'zoomcamp-llm-hw2' creada exitosamente.


In [54]:
points = []

for id, doc in enumerate(documents, 1):
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=f"{doc['question']} {doc['text']}", model=MODEL_SELECT),
            payload={
                "question": doc['question'],
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            }
        )
        points.append(point)

In [55]:
client.upsert(
    collection_name=COLLECTION_NAME,
    points=points,
    wait=True
)

print(f"{len(points)} puntos insertados en la colección usando 'upsert'.")

375 puntos insertados en la colección usando 'upsert'.


In [56]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=COLLECTION_NAME,
        query=models.Document(
            text=query,
            model=MODEL_SELECT 
        ),
        limit=limit,
        with_payload=True
    )

    return results

In [57]:
q = 'I just discovered the course. Can I join now?'
result = search(q)

In [58]:
result

QueryResponse(points=[ScoredPoint(id=15, version=1, score=0.8703172, payload={'question': 'The course has already started. Can I still join it?', 'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'mlops-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

In [59]:
result.points[0].score

0.8703172

In [60]:
result.points[0].payload['text']

'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.'

In [61]:
result.points[0].payload['question']

'The course has already started. Can I still join it?'